<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/langchain_anthropic_contextual_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -qU langchain langchain-community langchain_chroma langchain_openai langchain_cohere langchain-text-splitters pypdf chromadb

In [35]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

loader = PyPDFLoader("nio-q2-2024.pdf")
split_documents = loader.load_and_split(text_splitter=text_splitter)

In [36]:
len(split_documents)

21

In [28]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""

In [29]:
from langchain_core.prompts import PromptTemplate

In [30]:
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

In [31]:
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

In [32]:
anthropic_llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    model="anthropic/claude-3-haiku",
    max_tokens=1024,
    temperature=0,
    default_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
)

In [33]:
def process_documents(split_documents):
    all_content_in_batch = "\n".join([doc.page_content for doc in split_documents])

    # Create the system message
    system_message = SystemMessage([
        {"type": "text", "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=all_content_in_batch), "cache_control": {"type": "ephemeral"}}
    ])

    for current_doc in split_documents:
        messages = [
            system_message,
            HumanMessage([
                {"type": "text", "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=current_doc.page_content)}
            ])
        ]

        # Invoke the LLM
        response = anthropic_llm.invoke(messages)
        print(response.content)

        # Update the document's content
        current_doc.page_content = response.content + "\n\n" + current_doc.page_content

In [37]:
# Usage
process_documents(split_documents)

This chunk provides the key highlights of NIO Inc.'s unaudited financial results for the second quarter of 2024, including details on vehicle deliveries, revenue, gross profit, and net loss.
This chunk provides key financial results for NIO Inc. in the second quarter of 2024, including vehicle sales, vehicle margin, and cash and cash equivalents. It is part of the larger earnings report that discusses NIO's quarterly performance and business outlook.
This chunk provides key financial highlights and recent business developments for NIO Inc. in the second quarter of 2024, including record-breaking vehicle deliveries, strong financial performance, and the launch of the ONVO brand.
This chunk discusses NIO's financial results for the second quarter of 2024, including commentary from the CEO and CFO on the company's performance and outlook.
This chunk discusses the company's vehicle sales, other sales, cost of sales, gross profit, gross margin, and vehicle margin in the second quarter of 20

In [38]:
split_documents[0].page_content

"This chunk provides the key highlights of NIO Inc.'s unaudited financial results for the second quarter of 2024, including details on vehicle deliveries, revenue, gross profit, and net loss.\n\nNIO Inc. Reports Unaudited Second Quarter 2024 Financial Results\n09/5/2024\nQuarterly Total Revenues reached RMB17,446.0 million (US$2,400.6 million)i\nQuarterly Vehicle Deliveries were 57,373 units\nSHANGHAI, Sept. 05, 2024 (GLOBE NEWSWIRE) -- NIO Inc. (NYSE: NIO; HKEX: 9866; SGX: NIO) (“NIO” or the “Company”), a\npioneer and a leading company in the global smart electric vehicle market, today announced its unaudited financial results for the\nsecond quarter ended June 30, 2024.\nOperating Highlights for the Second Quarter of 2024\nVehicle deliveries  were 57,373 in the second quarter of 2024, consisting of 32,562 premium smart electric SUVs and\n24,811 premium smart electric sedans, representing an increase of 143.9% from the second quarter of 2023, and an\nincrease of 90.9% from the first q

In [39]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    split_documents,
    embedding=OpenAIEmbeddings(
        openai_api_key=userdata.get('OPENAI_API_KEY'),
        model="text-embedding-3-large"
    )
)

In [40]:
retriever = vectorstore.as_retriever()

In [42]:
from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

In [44]:
compressor = CohereRerank(
    model="rerank-english-v3.0",
    cohere_api_key=userdata.get('COHERE_API_KEY')
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [45]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [46]:
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | anthropic_llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [49]:
rag_chain.invoke("What is the vehicle sales?")

'Based on the provided context, the vehicle sales for NIO Inc. in the second quarter of 2024 were RMB15,679.6 million, representing an increase of 87.1% from the first quarter of 2024 and an increase of 118.2% from the second quarter of 2023.'